In [1]:
import h5py
import numpy as np
# import matplotlib.pyplot as plt
from tifffile import imwrite, imread, tiffcomment, TiffFile
from ome_types import from_tiff
from glob import glob
import os
import affine_matrices
from scipy.ndimage import affine_transform
from timeit import default_timer as timer

In [2]:
## define affine matrices for deskewing

def deskew(opm_angle, size_x, size_z):
    matrix_1_iso = affine_matrices.scale_matrix(sz=size_z/size_x)  # scale like in MVR calibration
    matrix_2_rot = affine_matrices.rotation_matrix(theta=np.pi*90/180, axis='x')  # make so stack direction is y not z(prime) like in lab
    matrix_3_scale = affine_matrices.scale_matrix(sz=np.sin(opm_angle))  # scale before shear to preserve aspect ratio
    matrix_4_shear = affine_matrices.skew_matrix(opm_angle)  # shear to deskew

    total_matrix = matrix_4_shear @ matrix_3_scale @ matrix_2_rot @ matrix_1_iso
    return total_matrix


In [4]:
tiff_data_dir = "C:/Users/lnr19/Documents/y_stage_scan_20241216-171205_c0p0/"
out_dir = "C:/Users/lnr19/Documents/y_stage_scan_20241216-171205_c0p0/deskewed/"
folder_name = "HamamatsuHam_DCAM-dopm"
# print(tiff_data_dir)
# print(os.listdir(tiff_data_dir))
tiff_dirs = glob(os.path.join(tiff_data_dir, "dOPM*"))
for tiff_dir in tiff_dirs:
    tiff_dir_basename = os.path.basename(tiff_dir)
    stack_dir = os.path.join(tiff_data_dir, tiff_dir, folder_name)  # dir containg the tiff stacks
    # print(stack_dir)
    stacks = glob(os.path.join(stack_dir, "*.tif"))
    print(stacks)
    omexml = from_tiff(stacks[0])
    iminfo = omexml.images[0]

    size_x = iminfo.pixels.size_x
    size_y = iminfo.pixels.size_y
    size_z = len(omexml.images)
    vox_size_x_um = iminfo.pixels.physical_size_x
    vox_size_y_um = iminfo.pixels.physical_size_y
    vox_size_z_um = iminfo.pixels.physical_size_z

    print("size_x", size_x)
    print("size_y", size_y)
    print("size_z", size_z)
    
    imstack = np.zeros((size_x, size_y, size_z), dtype=np.uint16)

    start_time = timer()

    offset = 0
    for i in range(len(stacks)):
        print("loading",  stacks[i])
        with TiffFile(stacks[i]) as tif:
            print(tif.pages)
            for n, page in enumerate(tif.pages):
                # print("page", n)
                imstack[:,:,n+offset] = page.asarray().T
        offset += n+1

    end_time = timer()
    print("Read time", (end_time-start_time), "ms")

    #imstack = imread(stacks[0])
    print(np.shape(imstack))
    affine_matrix = deskew(opm_angle=45*np.pi/180, size_x=size_x, size_z=size_z)
    start_time = timer()

    ## chunk the affines to improve performance, only in xy
    chunk_size_x = 100
    chunk_size_y = 100
    n_chunks_x = size_x//chunk_size_x
    n_chunks_y = size_y//chunk_size_y

    for i in range(n_chunks_x):
        for j in range(n_chunks_y):
            # counter for chunks
            print(f"chunk {i}, {j} out of {n_chunks_x}, {n_chunks_y}")
            # adjust chunk size for last chunk
            chunk_size_x_ = chunk_size_x if i < n_chunks_x-1 else size_x - i*chunk_size_x
            chunk_size_y_ = chunk_size_y if j < n_chunks_y-1 else size_y - j*chunk_size_y

            chunk = imstack[i*chunk_size_x_:(i+1)*chunk_size_x_, j*chunk_size_y_:(j+1)*chunk_size_y_, :]
            chunk_transformed = affine_transform(chunk, affine_matrix)
            imstack[i*chunk_size_x_:(i+1)*chunk_size_x_, j*chunk_size_y_:(j+1)*chunk_size_y_, :] = chunk_transformed

    imwrite(os.path.join(outdir,f"deskewed{tiff_dir_basename}.tif"), imstack)

    transformed = affine_transform(imstack, affine_matrix)
    end_time = timer()
    print("Transform time", (end_time-start_time), "ms")
    


    # affine_transform

    ## ome_types reads SizeX, SizeY, SizeZ as size_x etc., i.e. it converts from camel case to snake case
    print(iminfo)
    


['C:/Users/lnr19/Documents/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-dopm\\HamamatsuHam_DCAM-dopm_MMStack.ome.tif', 'C:/Users/lnr19/Documents/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-dopm\\HamamatsuHam_DCAM-dopm_MMStack_1.ome.tif', 'C:/Users/lnr19/Documents/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-dopm\\HamamatsuHam_DCAM-dopm_MMStack_2.ome.tif', 'C:/Users/lnr19/Documents/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-dopm\\HamamatsuHam_DCAM-dopm_MMStack_3.ome.tif']
size_x 2304
size_y 2304
size_z 1600
loading C:/Users/lnr19/Documents/y_stage_scan_20241216-171205_c0p0\dOPM_t0000_p0000_z0000_c0000_view1\HamamatsuHam_DCAM-dopm\HamamatsuHam_DCAM-dopm_MMStack.ome.tif
<tifffile.TiffPages @9950>
loading C:/Users/lnr19/Documents/y_stage_scan_20241216-171205_c0p0\dOPM_t0000_p0000_z0000_c0000_view1\HamamatsuHam_DCA

NameError: name 'chunk_size_xy' is not defined

In [39]:
affine_matrix = deskew(opm_angle=45*np.pi/180, size_x=size_x, size_z=size_z)
start_time = timer()
transformed = affine_transform(imstack[0:500,0:500,0:1600], affine_matrix)
end_time = timer()
print("Transform time", (end_time-start_time), "ms")


Transform time 1420.5133053999161 ms


In [40]:
imwrite("D:/y_stage_scan_20241216-171205_c0p0/transformed.tif", transformed, imagej=True, resolution=(1/vox_size_x_um, 1/vox_size_y_um), metadata={'spacing': vox_size_z_um, 'unit': 'um', 'axes': 'XYZ', 'hyperstack': 'true', 'mode': 'composite'})

In [16]:
dir(iminfo)
size_x = iminfo.pixels.size_x
size_y = iminfo.pixels.size_y
size_z = iminfo.pixels.size_z
vox_size_x = iminfo.pixels.physical_size_x
vox_size_y = iminfo.pixels.physical_size_y
vox_size_z = iminfo.pixels.physical_size_z

len(omexml.images)

1600

In [27]:
imread("D:\\testforpy.tif").shape   # z, y, x

(10, 50, 100)

In [25]:
imstack.shape

(1600, 2304, 2304)

In [2]:

skewmat = affine_matrices.skew_matrix(0,0.75)
print(skewmat)
np.linalg.det(skewmat)

[[1.   0.   0.   0.  ]
 [0.75 1.   0.   0.  ]
 [0.   0.   1.   0.  ]
 [0.   0.   0.   1.  ]]


np.float64(1.0)